# LR-torch版

In [1]:
import numpy as np
import pandas as pd
from plotly import graph_objects as go
import torch
from torch import nn
from torch.utils.data import TensorDataset,DataLoader

In [2]:
# 样本数
n = 400

# 生成数据集
X = 10 * torch.rand([n, 2]) - 5.
w0 = torch.tensor([[2.0], [-3.0]])
b0 = torch.tensor([[10.0]])
Y = X @ w0 + b0 + torch.normal(0., 2., size=[n, 1])

In [3]:
from plotly.subplots import make_subplots
# 预览
trace1 = go.Scatter(x=X[:, 0].numpy(), y=Y[:, 0], mode='markers', name='x',xaxis='x',yaxis='y')
trace2 = go.Scatter(x=X[:, 1].numpy(), y=Y[:, 0], mode='markers', name='x1',xaxis='x1',yaxis='y1')

fig = make_subplots(rows=1,  # 将画布分为两行
                    cols=2,  # 将画布分为两列
                    subplot_titles=["trace0的标题",
                                    "trace1的标题",
                                    ],  # 子图的标题
                    x_title="x",
                    y_title="y"
                    )
fig.add_trace(trace1,1,1)
fig.add_trace(trace2,1,2)
fig.show()

## 低阶api

In [4]:
# 构建数据迭代器
def data_iter(features: torch.Tensor, labels, batch_size=8):
    num_exampls = len(features)
    idx = list(range(num_exampls))
    np.random.shuffle(idx)
    for i in range(0, num_exampls, batch_size):
        new_idx = torch.LongTensor(idx[i:min(i + batch_size, num_exampls)])
        yield features.index_select(0, new_idx), labels.index_select(0, new_idx)


(features,labels) = next(data_iter(X,Y))
print('features: ',features)
print('labels: ',labels)

features:  tensor([[ 0.6172, -3.5332],
        [-4.4109, -0.1245],
        [ 3.7436, -1.0021],
        [ 0.6293, -3.4828],
        [ 1.5604, -4.1816],
        [-4.3760, -4.5093],
        [-2.0758, -0.2032],
        [ 0.8706, -3.6473]])
labels:  tensor([[23.5286],
        [ 1.2086],
        [21.0747],
        [24.5283],
        [24.3856],
        [10.8281],
        [ 9.4427],
        [23.5438]])


In [5]:
class LinearRegression:
    def __init__(self):
        self.w = torch.rand_like(w0, requires_grad=True)
        self.b = torch.zeros_like(b0, requires_grad=True)

    def forward(self, x):
        return x@self.w + self.b

    def loss_func(self, y_pred, y_true):
        return torch.mean(torch.pow((y_true-y_pred),2))

model = LinearRegression()

In [6]:
lr = 0.001
def train_step(model, features, labels):

    # 正向
    predictions = model.forward(features)
    # 损失函数
    loss = model.loss_func(predictions, labels)
    # 反向
    loss.backward()

    with torch.no_grad():
        # 更新参数
        model.w -= lr*model.w.grad
        model.b -= lr*model.b.grad

        #梯度清零
        model.w.grad.zero_()
        model.b.grad.zero_()
    return loss

In [7]:
# 测试一个batch
batch_size = 10
(features, labels) = next(data_iter(X, Y, batch_size))
train_step(model, features, labels)

tensor(164.1734, grad_fn=<MeanBackward0>)

In [8]:
# 完整训练
def train_model(model, epochs):
    for epoch in range(1,epochs+1):
        for features, labels in data_iter(X,Y,batch_size):
            loss = train_step(model, features, labels)

        if epoch % 200 == 0:
            print('='*66)
            print('epoch={}   loss={}'.format(epoch,loss.item()))
            print('model.w={}   model.b={}'.format(model.w.data, model.b.data))

train_model(model, epochs=1000)

epoch=200   loss=3.0922446250915527
model.w=tensor([[ 1.9875],
        [-3.0241]])   model.b=tensor([[9.9955]])
epoch=400   loss=1.5964200496673584
model.w=tensor([[ 1.9816],
        [-3.0276]])   model.b=tensor([[9.9957]])
epoch=600   loss=3.196270227432251
model.w=tensor([[ 1.9866],
        [-3.0205]])   model.b=tensor([[9.9961]])
epoch=800   loss=5.564911842346191
model.w=tensor([[ 1.9889],
        [-3.0223]])   model.b=tensor([[9.9965]])
epoch=1000   loss=4.4142608642578125
model.w=tensor([[ 1.9897],
        [-3.0271]])   model.b=tensor([[9.9958]])


In [9]:
# 结果预览

trace1 = go.Scatter(x=X[:, 0].numpy(), y=Y[:, 0], mode='markers', name='x',xaxis='x',yaxis='y')
trace2 = go.Scatter(x=X[:, 1].numpy(), y=Y[:, 0], mode='markers', name='x1',xaxis='x1',yaxis='y1')
trace3 = go.Scatter(x=X[:, 0],y=(X[:, 0]*model.w[0].data+model.b[0].data), name='line')
trace4 = go.Scatter(x=X[:, 1],y=(X[:, 1]*model.w[1].data+model.b[0].data), name='line')

fig = make_subplots(rows=1,  # 将画布分为两行
                    cols=2,  # 将画布分为两列
                    subplot_titles=["trace0的标题",
                                    "trace1的标题",
                                    ],  # 子图的标题
                    x_title="x",
                    y_title="y"
                    )
fig.add_trace(trace1,1,1)
fig.add_trace(trace2,1,2)
fig.add_trace(trace3,1,1)
fig.add_trace(trace4,1,2)
fig.show()

## 中阶api

In [10]:
print(X.shape)
print(Y.shape)
# 数据通道
ds = TensorDataset(X,Y)
dl = DataLoader(ds,batch_size=10,shuffle=True,)

torch.Size([400, 2])
torch.Size([400, 1])


In [11]:
# 模型
model = nn.Linear(2,1)
model.loss_func = nn.MSELoss()
model.optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

In [12]:
def train_step(model, features, labels):
    # 正向
    predictions = model(features)
    loss = model.loss_func(predictions, labels)

    # 反向
    loss.backward()
    
    model.optimizer.step()
    model.optimizer.zero_grad()
    return loss.item()

# 测试一个batch
features, labels = next(iter(dl))
train_step(model, features, labels)

276.5502014160156

In [13]:
def train_model(model, epochs):
    for epoch in range(1, epochs+1):
        for features, labels in dl:
            loss = train_step(model, features, labels)
        if epoch % 50 == 0:
            print('='*66)
            w = model.state_dict()['weight']
            b = model.state_dict()['bias']
            print('epoch=',epoch,'loss=',loss)
            print('w=',w)
            print('b=',b)
train_model(model, 250)

epoch= 50 loss= 2.643747329711914
w= tensor([[ 1.9747, -3.0410]])
b= tensor([9.9693])
epoch= 100 loss= 3.7125580310821533
w= tensor([[ 1.9963, -2.9695]])
b= tensor([9.9949])
epoch= 150 loss= 2.7248759269714355
w= tensor([[ 2.0205, -2.9615]])
b= tensor([10.0075])
epoch= 200 loss= 2.87420916557312
w= tensor([[ 1.9536, -3.0509]])
b= tensor([9.9946])
epoch= 250 loss= 2.4270541667938232
w= tensor([[ 2.0276, -3.0249]])
b= tensor([9.9928])


In [14]:
# 验证结果预览
w = model.state_dict()['weight']
b = model.state_dict()['bias']

trace1 = go.Scatter(x=X[:, 0].numpy(), y=Y[:, 0], mode='markers', name='x',xaxis='x',yaxis='y')
trace2 = go.Scatter(x=X[:, 1].numpy(), y=Y[:, 0], mode='markers', name='x1',xaxis='x1',yaxis='y1')
trace_mid3 = go.Scatter(x=X[:, 0],y=(X[:, 0]*w[0,0]+b[0]), name='line1')
trace_mid4 = go.Scatter(x=X[:, 1],y=(X[:, 1]*w[0,1]+b[0]), name='line2')

fig = make_subplots(rows=1,  # 将画布分为两行
                    cols=2,  # 将画布分为两列
                    subplot_titles=["trace0的标题",
                                    "trace1的标题",
                                    ],  # 子图的标题
                    x_title="x",
                    y_title="y"
                    )
fig.add_trace(trace1,1,1)
fig.add_trace(trace2,1,2)
fig.add_trace(trace_mid3,1,1)
fig.add_trace(trace_mid4,1,2)
fig.show()

## 高阶api

In [17]:
print(X.shape)
print(Y.shape)
# 数据通道
ds = TensorDataset(X,Y)
size_train, size_valid = int(400*0.7), 400-int(400*0.7)
ds_train, ds_valid = torch.utils.data.random_split(ds, [size_train, size_valid])

dl_train = DataLoader(ds_train, batch_size=10, shuffle=True, num_workers=2)
dl_valid = DataLoader(ds_valid, batch_size=10, num_workers=2)
dl_train

torch.Size([400, 2])
torch.Size([400, 1])


In [25]:
# !pip install torchkeras
from torchkeras import Model
class LR(Model):
    def __init__(self):
        super(LR, self).__init__()
        self.fc = nn.Linear(2,1)
    
    def forward(self, x):
        x = self.fc(x)
        return x

model = LR()

model.summary(input_shape=(2,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 1]               3
Total params: 3
Trainable params: 3
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.000008
Forward/backward pass size (MB): 0.000008
Params size (MB): 0.000011
Estimated Total Size (MB): 0.000027
----------------------------------------------------------------


In [29]:
def mean_absolute_error(y_pre, y_true):
    return torch.mean(torch.abs(y_pre-y_true))

def mean_absolute_percent_error(y_pred,y_true):
    absolute_percent_error = (torch.abs(y_pred-y_true)+1e-7)/ (torch.abs(y_true)+1e-7)
    return torch.mean(absolute_percent_error)

model.compile( loss_func=nn.MSELoss(),
        optimizer=torch.optim.Adam(model.parameters(),lr=0.01),
        metrics_dict={"mae":mean_absolute_error,"mape":mean_absolute_percent_error})

dfhistory = model.fit(200, dl_train=dl_train, dl_val=dl_valid, log_step_freq=20)

Start Training ...

================================================================================2021-11-30 06:46:12
{'step': 20, 'loss': 205.681, 'mae': 11.723, 'mape': 1.204}

 +-------+---------+--------+-------+----------+---------+----------+
| epoch |   loss  |  mae   |  mape | val_loss | val_mae | val_mape |
+-------+---------+--------+-------+----------+---------+----------+
|   1   | 206.706 | 11.826 | 1.143 | 179.138  |  11.139 |  1.002   |
+-------+---------+--------+-------+----------+---------+----------+

================================================================================2021-11-30 06:46:12
{'step': 20, 'loss': 186.278, 'mae': 11.346, 'mape': 1.059}

 +-------+---------+--------+-------+----------+---------+----------+
| epoch |   loss  |  mae   |  mape | val_loss | val_mae | val_mape |
+-------+---------+--------+-------+----------+---------+----------+
|   2   | 184.165 | 11.196 | 1.031 | 159.074  |  10.584 |   1.09   |
+-------+---------+--------+------

In [35]:
#验证
w,b = model.state_dict()["fc.weight"],model.state_dict()["fc.bias"]

trace1 = go.Scatter(x=X[:, 0].numpy(), y=Y[:, 0], mode='markers', name='x',xaxis='x',yaxis='y')
trace2 = go.Scatter(x=X[:, 1].numpy(), y=Y[:, 0], mode='markers', name='x1',xaxis='x1',yaxis='y1')
trace_hig3 = go.Scatter(x=X[:, 0],y=(X[:, 0]*w[0,0]+b[0]), name='line1')
trace_hig4 = go.Scatter(x=X[:, 1],y=(X[:, 1]*w[0,1]+b[0]), name='line2')

fig = make_subplots(rows=1,  # 将画布分为两行
                    cols=2,  # 将画布分为两列
                    subplot_titles=["trace0的标题",
                                    "trace1的标题",
                                    ],  # 子图的标题
                    x_title="x",
                    y_title="y"
                    )
fig.add_trace(trace1,1,1)
fig.add_trace(trace2,1,2)
fig.add_trace(trace_hig3,1,1)
fig.add_trace(trace_hig4,1,2)
fig.show()

In [38]:
# 模型评估
dfhistory

,loss,mae,mape,val_loss,val_mae,val_mape
0,206.705819,11.826292,1.142728,179.138465,11.139186,1.001967
1,184.164596,11.195637,1.030634,159.073689,10.583729,1.090158
2,163.851413,10.594002,1.078079,141.665758,10.066929,1.199580
3,146.259105,10.017008,1.242281,126.222911,9.563424,1.292788
4,130.541949,9.522455,1.389290,113.160712,9.121043,1.379391
...,...,...,...,...,...,...
195,3.619958,1.487968,0.504868,2.830442,1.329270,0.554209
196,3.609753,1.491666,0.520017,2.856388,1.334395,0.549097
197,3.608472,1.489018,0.510647,2.830187,1.333894,0.559417
198,3.613289,1.482289,0.500746,2.876758,1.345900,0.564827


In [48]:
from plotly import graph_objects as go
def fig_go(dfhistory, metric):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dfhistory.index, y=dfhistory[metric], name='train_'+metric))
    fig.add_trace(go.Scatter(x=dfhistory.index, y=dfhistory['val_'+metric], name='val_'+metric))
    fig.update_xaxes(title_text="epochs")
    fig.update_yaxes(title_text=metric)
    fig.show()


In [49]:
fig_go(dfhistory, 'loss')
fig_go(dfhistory, 'mae')
fig_go(dfhistory, 'mape')

In [50]:
# 评估模型
model.evaluate(dl_valid)

{'val_loss': 2.81837605436643,
 'val_mae': 1.3285805433988571,
 'val_mape': 0.5507563116649786}

In [51]:
# 使用模型
# 预测
dl = DataLoader(TensorDataset(X))
model.predict(dl)[0:10]

tensor([[13.7463],
        [-5.7723],
        [22.0804],
        [23.5855],
        [19.9094],
        [22.0829],
        [ 0.6314],
        [ 9.2593],
        [-3.0646],
        [33.8110]])